# Measurements and Analisys of Methane in Colombia


In [11]:
# Import all libraries

import os
os.environ['USE_PYGEOS'] = '0'
import ee
import geemap
import geopandas as gdp
from ipyleaflet import GeoJSON
from zipfile import ZipFile
from geeS2downloader import GEES2Downloader
import numpy as np


# Initializate GEE API
ee.Initialize()

### 1. Defining wish satellite is to be used and their input parameters



In [12]:
satellite = 'COPERNICUS/S5P/OFFL/L3_CH4'
layer = 'CH4_column_volume_mixing_ratio_dry_air_bias_corrected'
initDate = '2018-05-31'
finalDate = '2022-12-31'


### 2. Importing all shp files and satellite images

In [13]:
def getImageCollection(satellite, layer, initDate, finalDate):
    
    imageCollection = ee.ImageCollection(satellite).select(layer).filterDate(initDate, finalDate)
    return imageCollection

fronterasMaritimasCol = ee.FeatureCollection('projects/ee-jolejua/assets/EEZ_land_union_v3_202003')
fronterasMaritimasCol = fronterasMaritimasCol.filter(ee.Filter.eq('UNION', 'Colombia'))


methaneImageCollection = getImageCollection(satellite, layer, initDate, finalDate)

### 3. Clip image of interest to colombia shape

In [14]:
methaneImageClip = methaneImageCollection.mean()
methaneImageClip = methaneImageClip.clip(fronterasMaritimasCol)


### 4. Model training

In [5]:
methaneImageCollectionClip = ee.ImageCollection.fromImages([methaneImageClip]) # from Image to ImageCollection

pixelsSample = methaneImageClip.sample(region = fronterasMaritimasCol.geometry(), scale=5000, factor = 1, geometries=True) # generate a sample of the pixels in the images

n_clusters = 3
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(pixelsSample)
result = methaneImageClip.cluster(clusterer)


In [6]:
Map = geemap.Map(center=(4.6,-74),zoom=9)
Map.addLayer(result.randomVisualizer(), {}, 'clusters')
Map

EEException: Computation timed out.

### 5. Means specifics data per year or month

In [17]:
from datetime import datetime
import numpy as np
from dateutil.relativedelta import relativedelta




def getImageRange(satellite, layer, startDate, endDate):
    
    date_format = '%Y-%m-%d'
    endDate = datetime.strptime(endDate, date_format)
    startDate = datetime.strptime(startDate, date_format)
    difference = relativedelta(endDate, startDate)
    date_i = endDate + relativedelta(months=1)
    months = difference.years * 12 + difference.months
    
    imagenes = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
    
    for i in range(months):
        
        satRowImage = ee.ImageCollection(satellite).select(layer).filterDate(startDate, date_i)
        meanSatImage = satRowImage.mean()
        clipSatImage = meanSatImage.clip(fronterasMaritimasCol)
        
        
        startDate = date_i
        newEndDate = date_i + relativedelta(months=1)
        
        endDate = newEndDate
        print(endDate)
        
    return imagenes, date
s_date = '2018-05-31' #Primera fecha disponible en el satelite
e_date='2022-04-19'
imagenes, date = getImageRange(satellite, layer, s_date, e_date)

2022-06-19 00:00:00


AttributeError: 'datetime.datetime' object has no attribute 'split'

In [9]:
collection = ee.ImageCollection.fromImages(imagenes)
vis_params = {
    'min': 1600, 
    'max': 2000,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
    'opacity': 0.8
}
#Arreglar límites para la emisión
image = ee.Image(collection.mean())
Mapg = geemap.Map(center=(4.6,-74),zoom=5.3)
Mapg.addLayer(image, vis_params, 'First image',True,0.9)
Mapg


Map(center=[4.6, -74], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…